In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy.coordinates import SkyCoord
from astropy import units
from astroquery.skyview import SkyView
import numpy as np
from astropy import units as u

In [6]:
df1=pd.read_csv('galaxias_2.csv')
display(df1.head(2))
display(df1.shape)
data=df1

,objID,ra,dec,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,petroR90_r,Color,elliptical,spiral,uncertain,distancia_L,Mag_abs
0,1.237674e+18,119.822479,42.008528,17.36539,15.46586,14.54658,14.07490,13.69937,18.121160,-2.818807,1.0,0.0,0.0,135.131916,-21.191136
1,1.237674e+18,118.185239,33.699089,19.95136,18.35397,17.54043,17.05026,16.71222,3.657189,-2.410933,0.0,0.0,1.0,153.893111,-18.473102


(32623, 15)

In [4]:
from math import sqrt
from joblib import Parallel, delayed
%time
Parallel(n_jobs=2)(delayed(sqrt)(i**2) for i in range(10))

CPU times: total: 0 ns
Wall time: 0 ns


[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [7]:
coords_gx = SkyCoord(data.ra.iloc[2], data.dec.iloc[2], unit=(u.deg, u.deg))
img = SkyView.get_images(position=coords_gx, survey=['SDSSg', 'SDSSi', 'SDSSr', 'SDSSu', 'SDSSz'])
display(len(img))
display(img[0].info())
display(img[0][0].shape)

5

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     122   (300, 300)   float32   


None

(300, 300)

In [8]:
def image_transformation(ra, dec):
    coords_gx = SkyCoord(ra, dec, unit=(u.deg, u.deg))
    img = SkyView.get_images(position=coords_gx, survey=['SDSSg', 'SDSSi', 'SDSSr', 'SDSSu', 'SDSSz'])
    SDSSg = img[0][0].data.flatten()
    SDSSi = img[0][0].data.flatten()
    SDSSr = img[0][0].data.flatten()
    SDSSu = img[0][0].data.flatten()
    SDSSz = img[0][0].data.flatten()

    result = np.concatenate([SDSSg,SDSSi,SDSSr,SDSSu,SDSSz])

    return result

In [11]:
stack = image_transformation(df1.ra[0], df1.dec[0])
%time
for index, row in df1.iterrows():
    if index>0 and index<100:
        print(index)
    temp = Parallel(n_jobs=4)(delayed(image_transformation)(row.ra, row.dec)
    stack = np.vstack([stack,temp])
    df2 = pd.DataFrame(stack)
    #df2.to_csv('df2.csv')

SyntaxError: '(' was never closed (334331138.py, line 6)

In [ ]:
df2